# Primer capitulo, entender el problema

## Que genera run_bulk_simulations.py?



# Archivos generados por run_bulk_simulations.py

Cuando ejecutamos el script `run_bulk_simulations.py`, se crea una estructura de directorios dentro de `data/policies/{POLICY_NAME}/experiments/{timestamp}/` que contiene varios archivos destinados a almacenar y analizar los resultados de múltiples simulaciones. Los principales archivos generados son:

## Archivos de Datos Sin Procesar

- **raw_data.json**: Contiene todos los datos sin procesar de cada simulación individual. Incluye detalles de cada ejecución (run_id, city_size, success, path_length, time_taken), datos de recursos (allocated, used, needed), proxy_data (indicadores ambientales) y las decisiones de la política. Estos datos se organizan tanto en una lista general como agrupados por tamaño de ciudad.

## Archivos de Métricas Agregadas

- **core_metrics.json**: Contiene métricas clave del experimento agregadas, incluyendo:
  - Metadata (nombre de la política, ID del experimento, timestamp)
  - Rendimiento general (tasa de éxito, tiempo promedio, longitud promedio de ruta)
  - Detalles de recursos (promedio asignado, usado y eficiencia por tipo de recurso)
  - Métricas proxy (estadísticas de indicadores ambientales)
  - Desgloses por tamaño de ciudad

- **core_metrics.csv**: Versión en formato CSV del archivo anterior, con nombres de columnas aplanados para facilitar el análisis en herramientas como pandas o Excel.

- **resource_metrics.json**: Métricas detalladas sobre el uso de recursos, incluyendo:
  - Promedios por tipo de recurso (explosives, ammo, radiation_suits)
  - Eficiencia de uso
  - Análisis de recursos más utilizados y necesitados
  - Desglose por tamaño de ciudad

- **environmental_metrics.json**: Métricas sobre factores ambientales y su impacto:
  - Promedios de indicadores ambientales
  - Desglose por tamaño de ciudad
  - Correlaciones con el éxito de la misión

## Métricas por Ciudad Individual

En el directorio `cities/`:

- **city_{run_id}_metrics.json**: Archivo para cada ciudad simulada con métricas detalladas específicas
- **city_{run_id}_metrics.csv**: Versión CSV del archivo anterior

## Visualizaciones 

En el directorio `visualizations/`:

- **key_metrics.png**: Dashboard con distribución de resultados, tiempos y uso de recursos
- **success_rates.png**: Tasas de éxito por tamaño de ciudad
- **resource_efficiency.png**: Comparación de recursos asignados vs utilizados
- **environmental_impact.png**: Correlaciones entre factores ambientales y éxito de misión
- **performance_metrics.png**: Tabla resumen de indicadores clave
- **resource_impact.png**: Análisis de eficiencia de recursos
- **environmental_correlations.png**: Correlaciones entre diferentes factores ambientales

Estos archivos proporcionan una base completa para analizar el rendimiento de nuestra política de evacuación, identificar patrones y áreas de mejora a través de múltiples simulaciones y diferentes escenarios de ciudad.

# Acordeón: Elementos de las Ciudades en el Zombie Extraction Simulator

<details>
<summary><b>📊 Estructura Básica de la Ciudad</b></summary>

La ciudad se representa como un grafo no dirigido (`nx.Graph`) con las siguientes propiedades:

- **Nodos (nodes)**: Representan ubicaciones específicas dentro de la ciudad
- **Aristas (edges)**: Representan las conexiones o calles entre ubicaciones
- **Nodo inicial (starting_node)**: Punto de partida para la evacuación
- **Nodos de extracción (extraction_nodes)**: Destinos posibles para completar la evacuación
- **Pesos (weights)**: Distancias o dificultad para moverse entre nodos

La estructura `city.graph` de NetworkX permite aplicar algoritmos incorporados como búsqueda de camino más corto (`nx.shortest_path`).
</details>

<details>
<summary><b>🔍 Indicadores Proxy de Nodos</b></summary>

Los indicadores proxy para los nodos representan las condiciones ambientales en cada ubicación, en un rango de 0 a 1:

| Indicador | Descripción | Impacto |
|-----------|-------------|---------|
| **seismic_activity** | Vibraciones causadas por colapsos estructurales | Valores cercanos a 1 indican riesgo de colapsos catastróficos |
| **radiation_readings** | Niveles de radiación en la zona | Valores altos requieren trajes de protección radiológica |
| **population_density** | Concentración de actividad en el área | Valores altos pueden indicar supervivientes o infectados |
| **emergency_calls** | Señales de socorro recibidas | Picos pueden indicar oportunidades de rescate o zonas de peligro |
| **thermal_readings** | Actividad térmica detectada | Valores intermedios pueden revelar patrones de infectados |
| **signal_strength** | Calidad de la comunicación | 0 indica pérdida total de comunicación, 1 indica comunicación perfecta |
| **structural_integrity** | Estabilidad de edificios y estructuras | 0 indica colapso inminente, 1 indica estructuras intactas |

Estos datos están disponibles en `proxy_data.node_data[node_id]` como un diccionario de indicadores para cada nodo.
</details>

<details>
<summary><b>🛣️ Indicadores Proxy de Aristas</b></summary>

Los indicadores proxy para las aristas representan las condiciones de las rutas entre ubicaciones, en un rango de 0 a 1:

| Indicador | Descripción | Impacto |
|-----------|-------------|---------|
| **structural_damage** | Daño en la ruta o calle | 0 indica ruta despejada, 1 indica bloqueo total que requiere explosivos |
| **signal_interference** | Degradación de comunicaciones | 0 indica transmisión perfecta, 1 indica interferencia total |
| **movement_sightings** | Actividad detectada en la ruta | Valores altos indican mayor actividad (supervivientes o infectados) |
| **debris_density** | Acumulación de obstáculos | Valores altos indican mayor cantidad de escombros que dificultan el paso |
| **hazard_gradient** | Cambios en condiciones ambientales | 0 indica condiciones uniformes, 1 indica cambios abruptos y peligrosos |

Estos datos están disponibles en `proxy_data.edge_data[(node1, node2)]` como un diccionario de indicadores para cada arista.
</details>

<details>
<summary><b>🧰 Recursos Disponibles</b></summary>

Tu equipo puede llevar recursos limitados para enfrentar diferentes obstáculos:

| Recurso | Uso | Desafío relacionado |
|---------|-----|---------------------|
| **explosives** | Despejar rutas bloqueadas | Alto structural_damage en aristas |
| **ammo** | Enfrentar hordas de infectados | Alto thermal_readings en nodos (zombies) |
| **radiation_suits** | Protección contra radiación | Alto radiation_readings en nodos |

La cantidad total de recursos está limitada por `max_resources` y se debe distribuir estratégicamente entre los tres tipos.
</details>

<details>
<summary><b>📊 Interpretación de los Datos Proxy</b></summary>

### Lectura de los indicadores

Todos los indicadores están normalizados en una escala de 0 a 1:
- **0**: Ausencia total del fenómeno medido
- **1**: Máxima intensidad registrada hasta la fecha

### Correlación con obstáculos

Los datos de los sensores brindan pistas sobre los obstáculos que encontrarás:

1. **Obstáculos de radiación**:
   - Altos valores de `radiation_readings` (>0.35) indican zonas que requieren trajes de radiación
   - Se relacionan con `signal_interference` alta en aristas cercanas

2. **Obstáculos de zombies/infectados**:
   - Valores medios-altos de `thermal_readings` (>0.45) sugieren presencia de infectados
   - Se correlacionan con `movement_sightings` elevados en rutas cercanas

3. **Bloqueos de ruta**:
   - Valores altos de `structural_damage` (>0.4) indican rutas bloqueadas que necesitan explosivos
   - Suelen correlacionarse con `seismic_activity` elevada en nodos cercanos

4. **Lectura holística**:
   - Los expertos recomiendan no confiar en un solo indicador
   - La combinación de múltiples indicadores ofrece una imagen más completa
   - Los cambios abruptos en las lecturas (`hazard_gradient` alto) suelen ser más peligrosos que valores consistentemente altos
</details>

<details>
<summary><b>🎯 Condiciones de Éxito y Fracaso</b></summary>

### Para que una misión sea exitosa:

1. **Ruta válida**: El camino debe:
   - Comenzar en el nodo inicial (`city.starting_node`)
   - Terminar en uno de los nodos de extracción (`city.extraction_nodes`)
   - Tener conexiones válidas entre nodos consecutivos
   
2. **Recursos adecuados**:
   - Suficientes explosivos para despejar rutas bloqueadas
   - Suficiente munición para enfrentar a los infectados
   - Suficientes trajes para atravesar zonas radiactivas
   - Total de recursos ≤ `max_resources`

### Causas comunes de fracaso:

1. **Falta de recursos** durante la misión
2. **Ruta inválida** o imposible entre inicio y extracción
3. **Asignación incorrecta de recursos** (total > `max_resources`)
</details>

<details>
<summary><b>🔢 Métricas de Evaluación</b></summary>

Las simulaciones evalúan varias métricas para medir la efectividad de tu política:

1. **Tasa de éxito**: Porcentaje de misiones completadas exitosamente
2. **Tiempo promedio**: Duración media de las misiones
3. **Longitud de ruta**: Distancia total recorrida
4. **Eficiencia de recursos**:
   - Recursos asignados vs. utilizados
   - Eficiencia por tipo de recurso

Estas métricas se calculan tanto para experimentos individuales como agregadas para múltiples simulaciones.
</details>

## Que hace solution.py por default?

En este momento, el archivo `solution.py` implementa una estrategia muy básica por defecto. Veamos lo que está haciendo:

El archivo contiene una clase `EvacuationPolicy` con cuatro políticas diferentes que se pueden seleccionar cambiando el valor de `self.policy_type`:

1. Actualmente, está configurado para usar `policy_1` como se ve en la línea:
   ```python
   self.policy_type = "policy_1"  # Política por defecto
   ```

2. La implementación de `_policy_1` (línea 68-89) es muy simple:
   - Toma el primer nodo de extracción disponible como destino
   - Intenta calcular el camino más corto entre el nodo inicial y el nodo de extracción usando `nx.shortest_path`
   - Si no encuentra un camino, devuelve solo el nodo inicial
   - Distribuye los recursos disponibles de manera uniforme entre los tres tipos (explosivos, munición y trajes de radiación)

Específicamente, divide los recursos disponibles en tres partes iguales:
```python
resources = {
    'explosives': max_resources // 3,
    'ammo': max_resources // 3,
    'radiation_suits': max_resources // 3
}
```

Esta implementación es muy básica y no toma en cuenta:
- Las condiciones ambientales (no usa los datos de proxy)
- La distribución óptima de recursos según los obstáculos esperados
- Rutas alternativas que podrían ser más seguras

Las otras políticas (`_policy_2`, `_policy_3`, y `_policy_4`) están definidas pero tienen el mismo comportamiento básico por ahora, y están esperando ser implementadas con estrategias más avanzadas según las restricciones y capacidades definidas en el ejercicio.

Cuando empieza la simulación y tu código `solution.py` se ejecuta, tienes acceso a los siguientes datos de entrada:

### 1. Información del Grafo de la Ciudad (`city`)

- `city.graph`: Un objeto `networkx.Graph` que representa la estructura de la ciudad.
  - Contiene nodos y aristas con sus conexiones
  - Cada nodo tiene posiciones (coordenadas) accesibles mediante `city.graph.nodes[node]['pos']` 
  - Cada arista tiene un peso (distancia) accesible mediante `city.graph[node1][node2]['weight']`

- `city.starting_node`: El nodo de inicio donde está el equipo que debe ser evacuado.

- `city.extraction_nodes`: Una lista de nodos que son puntos de extracción válidos. Tu equipo debe llegar a cualquiera de estos nodos para completar la misión.

### 2. Datos del Proxy (`proxy_data`)

- `proxy_data.node_data`: Un diccionario que contiene indicadores ambientales para cada nodo.
  - Formato: `{node_id: {'indicator_name': value, ...}, ...}`
  - Indicadores incluyen: `seismic_activity`, `radiation_readings`, `population_density`, `emergency_calls`, `thermal_readings`, `signal_strength`, `structural_integrity`

- `proxy_data.edge_data`: Un diccionario que contiene indicadores ambientales para cada arista.
  - Formato: `{(node1, node2): {'indicator_name': value, ...}, ...}`
  - Indicadores incluyen: `structural_damage`, `signal_interference`, `movement_sightings`, `debris_density`, `hazard_gradient`

### 3. Recursos Disponibles

- `max_resources`: Un entero que representa el máximo total de recursos que puedes asignar para la misión. Estos recursos deben dividirse entre:
  - `explosives`: Para despejar rutas bloqueadas
  - `ammo`: Para enfrentar enemigos (zombies)
  - `radiation_suits`: Para protección contra radiación

### Estructura de las Políticas

El código `solution.py` tiene cuatro diferentes políticas que implementar:

1. **Policy 1**: Solo permite usar información básica del grafo (nodos, aristas, pesos), sin usar los datos de proxy.

2. **Policy 2**: Permite usar información del grafo y datos de proxy, pero no datos de simulaciones previas.

3. **Policy 3**: Permite usar toda la información anterior más datos históricos de simulaciones previas.

4. **Policy 4**: Sin restricciones, puedes usar cualquier enfoque.

### Lo que debes devolver

Tu función `plan_evacuation` debe devolver un objeto `PolicyResult` que contiene:

1. `path`: Una lista de IDs de nodos que forman la ruta de evacuación, comenzando con `city.starting_node` y terminando en algún nodo de `city.extraction_nodes`.

2. `resources`: Un diccionario que especifica cuántos recursos de cada tipo llevar:
   ```python
   {
       'explosives': x,
       'ammo': y,
       'radiation_suits': z
   }
   ```
   donde x + y + z <= max_resources

Para cada política, tendrás que implementar una estrategia diferente considerando las restricciones específicas de esa política.

### Comentarios adicionales

- La información que tienes es imperfecta - los indicadores de proxy son aproximaciones de las condiciones reales.
- Deberás determinar qué indicadores son más confiables para predecir obstáculos y cómo balancear la asignación de recursos.
- La selección de la ruta y la asignación de recursos están interrelacionadas - una ruta más larga pero "segura" podría requerir menos recursos que una ruta corta pero peligrosa.

¿Hay alguna parte específica de esta información que te gustaría que explicara con más detalle?